# Question and Answer assignment

## Task overview

### Formalization and definitions
_(Adapted from the assignment's description.)_

In this project, we will be focusing on one of the main features of Assist AI's technology: __macro suggestions__. Macro suggestions consist in recommending potential answers for a question. Throughout this report, the terms *__macro__* and *__template__* will be used as synonyms unless noted otherwise.

Assist AI currently approaches this problem as a prediction task. More specifically, answers to questions are extracted from historical data consisting of Customer Support tickets. In its simplest implementation, this kind of dataset will contain some large number of pairings of a question and the correct answer to that question according to some human annotator. Let's call this dataset D, such that D = List: ( _question_,    _answer_ ).

We also need to assume the use of some similarity metric that allows the system to judge how similar a question and an answer are, i.e., some quantitative indication of how similar the unanswered question is with respect to a previously answered question, so that the latter's answer can also be used as a reply to the former. The rationale behind this is the following: given dataset D and input question _q_ below

In [6]:
D_prime = [
    ('when was America discovered?', 'America was discovered in 1492'),
    ('what is the color of the sky?', 'The sky is blue'),
]

q = 'what year was America discovered?'

the system should be able to recognize that D' contains a pair whose first element has a high similarity with _q_ and its answer may therefore be used as the answer for _q_, too.

Besides answering previously unseen questions with information from observed similar questions, this type of predictive model can also be used to determine the similarity between a new answer from the ticket history and previous answers, in order to assess e.g. if we are in front of a new answer that requires building a new template. For illustration, let's assume we expanded dataset D' and added an item as shown below:

In [7]:
D_prime = [
    ('when was America discovered?', 'America was discovered in 1492'),
    ('what is the color of the sky?', 'The sky is blue')
]

#    Below is a hypothetical new ticket generated by a customer support representative.
#    The new ticket provides a new potential <quesiton, answer> template to be learned
#    by the system:
new_ticket = (                                     
    'when was the discovery of America?',          # new candidate question
    'Europeans first set foot on America in 1492'  # new candidate answer
)

In this case, we would like the similarity metric to reflect the fact that, from a semantic point of view, the answer in the first item of D' is a paraphrase of the question in the new ticket and, therefore, that no new template should be added for the question 'when was the discovery of America?'.

In this type of framework, the choice of similarity metric is very likely to be a key factor in the overall performance of the model.

### 1st task
On the basis of the problem description in the previous point, we will first address this second issue, namely, how to determine whether an input answer is similar to any of the answers in the templates already known by them system and, if so, which of those templates is the best match for the new question.

### 2nd task
In the second task, we will address the issue of assigning a relevant answer to a new question not previously observed in the customer support tickets from which the templates were originally extracted.

> Describe one method you would explore to create a classifier that can label new, previously unseen questions.

For this second part of the project, the task specifications state that we can assume the training set to contain:
1. between 1k and 100k instances, to be denoted as _|S|_.
2. and between 10 and 1k classes, to be denoted as _|C|_.

In our implementation:
* _|S|_ is set to 174 (all the instances in our dataset of questions and answers).
* _|C|_ is set to 174 (ibidem).
* For |S| = 174, a complete cycle of training and testing takes between 1.17 and 0.5 seconds, depending mainly on the features used.
* In its current implementation, the algorithm's training performance is approximately linear and runs in O(N) time. Generalizing from the performance observed with the available dataset (1.17 seconds/run), for a value of |S| = 10k we can estimate a running time slightly over 60 seconds.

## Challenges
#### The problem of thresholding
In order to assess the similarity between a new expression and previously observed expressions, one of the main challenges is choosing the right similarity threshold below which the two expressions are considered dissimilar and can therefore be reliably discarded as a match by the system.

In this case, the problem lies in the fact that potentially many pairs of expressions will be similar based on some of the features used by the system to establish the similarity between them. Therefore, an heuristic like picking always the highest-scoring match will usually result in all correct matches being selected but also in many incorrect matches being selected as well (since there is always a highest-scoring match, even when the actual score is not objectively high). That is, given that the similarity scale expresses relative similarity rather than absolute similarity, we need some other way for the system to be able to detect __true negatives__ with some degree of success.

In section ###, we will discuss in detail the implementation of a solution for this problem.

#### The problem of partial matches
Another issue, closely related to the problem of thresholding introduced in the previous point, is the fact that the system may miss a legitimate match between two answers due to the match existing between some of the answers' subunits rather than the full answers themselves.

That is, in order to measure the similarity between a pair of answers, the system needs to take into account some ratio of the number of features that they share with respect to their total number of features (by using e.g. the Euclidean distance or some similar metric). By definition, partial matches contain all the information relevant for triggering a match, plus some amount of irrelevant information that should not trigger it. In case that the features activated by the irrelevant information outweight the features activated by the relevant information, the similarity between the two may fall below the chosen threshold, resulting in a __false negative__. 

Details on a potential solution for this problem are given in section ###.

#### The problem of linguistic variation
Ultimately, linguistic variation is the key linguistic challenge underlying both tasks. That is, given that natural languages allow the same thought to be expressed in a number of different ways, any similarity metric aimed at measuring the semantic relatedness between two expressions must ideally provide robustness across all, or as many of their variants as possible, for optimal performance.

This type of robustness is determined by the choice of features, which must be consistent and must remain the same across variants of the same idea, i.e., feature extraction must be performed in such a way that the features extracted from two variants expressing the same overall meaning, must reflect none of the linguistic variations making the original expressions different.

The details of our approach for dealing with this phenomenon are discussed in section ####.

## Dataset
For our experiments, we will be using a subset of Microsoft's [WikiQA corpus](https://www.microsoft.com/en-us/research/publication/wikiqa-a-challenge-dataset-for-open-domain-question-answering/). More specifically, we will be using a subset of 174 instances for training, plus a variable number of additional instances for testing (ranging from 68 to 140 based on different parameterizations).

All instances in our final dataset were selected because they correspond to questions with at least 2 manually assigned correct answers in the WikiQA dataset. This provides us with enough data for tasks 1 and 2:
* Given that the dataset contains two answers for each question, we can use one of the answers to train the system, then have the algorithm predict the label for the other one. Since we know in advance that both are valid answers for the same question, we know what is the correct prediction we expect from the system. This is the data necessary for our first prediction task, in which we want to be able to determine the similarity between two answers.
* Given that the dataset also contains the question that triggers each answer, it also provides us with the data necessary for our second prediction task: suggesting a potential answer for a previously unseen question. Since the questions are never used in training, we could actually use all of them for testing but, for simplicity, we will use the same training and testing data size as for task 1.

Approximately ~210 instances in this dataset fulfill our requirement of having at least 2 correct answers that have been manually assigned.

## Approach
### Relevance filter versus Logistic Regression
For my initial experiments, I used a Logistic Regression (LR) classifier for the first task. However, the results were rather poor. After performing the error analysis, I noticed some trivial issues that could be fixed _post hoc_ with an inverse probability relevance filter (i.e., a process that only accepted a bag-of-words-based match between two items if the words in the intersection of the items' bags also happen to be the highest relevance/highest inverse-probability words in their respective bags).

However, if that kind of filter was implemented, it would already be able to handle the classification task itself too (as a mere lookup over the inverted index), making the LR classifier redundant to a certain extent. Based on grounds both of implementation ease and theoretical elegance, I finally decided not to use the classifier and rely solely on the relevance filter for the time being (where the relevance filter can be seen as an inverted index mapping words to answers containing those words, and their inverse probabilities for TF-IDF-like feature weighting).

- First of all, a reply doesn't necessarily belong to any templated answer class. __True negatives__
- In other cases, only part of the reply will contain a templated response. __By definition__ __Single questions, not mixed in my task. If they were, the similarity with respect some specific part of the answer should be higher than the similarity with respect to the whole__

### Features
We will consider a relatively simple set of features for our model:

* word n-grams
* character n-grams
* word s-skip-n-grams

## Results
### 1st versus 2nd task

## Error analysis


## Discussion and next steps
[From 1st task] __TO-DO: word embeddings, normalization__
[From 1st task] __TO-DO: word embeddings, concept expansion__
### Model enhancements
#### Question types and expected answers
PLACE, TIME
__Topic labels__ already handled by the system



